In [19]:
from Utilities import Mytokenizer
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy,math
import Transformer

In [20]:
train_path = '/home/jovyan/input/anki2023_en_ch/train.txt'
data_path = '/home/jovyan/input/anki2023_en_ch/cmn.txt'
tokenizer = Mytokenizer(data_path,'en')

中文字典字数 3643
英文字典字数 8349


## 自定义的数据集
这个数据集实现了将文字转id 并且每个过程都有对应的输出

In [21]:
class Mydataset(Dataset):
    """
    @file_path 数据存储位置
    @tokenizer 将文字id化的实例化后的Tokenizer
    @文件中 trg数据的位置
    
    由于Decoder的输入要求，一句话应被切分成多段，目标词数有多少就应切分多少次
    那么用一个自己写在Transformer中的batch类封装一句话，会自动的给句子mask，
    并且切分Decoder的输入输出,但是注意例如:
    trg为 "I love food" 那么应该输入四次，输出四次
        输入                   输出
    <BOS> mask mask mask   I 
    <BOS> I    mask mask   I love
    <BOS> I    love mask   I love food
    <BOS> I    love food   I love food <EOS>
    
    文件中的样子如下，中译英任务英文为目标语言 trg_index应为0
    Hi	嗨
    Hi	你 好
    Run	你 用 跑 的
    """
    def __init__(self,file_path,tokenizer,trg_index=0):
        self.tokenizer = tokenizer
        #读取所有文本
        with open(file_path,'r',encoding='utf8') as f:
            self.lines = f.readlines()
        #self.trg_count_words_line = []
        self.length = len(self.lines)
        self.trg_index = trg_index
        if trg_index==0:
            self.src_index = 1
        else:
            self.src_index = 0
        
    def __getitem__(self,index):
        src = self.lines[index].split('\t')[self.src_index]
        src = src.split('\n')[0]
        print("src:",src)
        trg = self.lines[index].split('\t')[self.trg_index]
        print("trg:",trg)
        #如上面的例子 三个词的句子应有四个样本,所以应该拷贝三次
        copy_time = len(trg.split(' '))
        print(copy_time)
        # src_id化 这里简单定义了src使用中文
        src_id = self.tokenizer.ch_token_id([src],len(src.split(' ')))
        print("src_id:",src_id)
        trg_id = self.tokenizer.en_token_id([trg],len(trg.split(' '))+2)
        print("trg_id:",trg_id)
        src_tensor = torch.LongTensor(src_id)
        trg_tensor = torch.LongTensor(trg_id)
        print('src_tensor:',src_tensor.shape,'trg_tensor:',trg_tensor.shape)
        #复制   
        #src_tensor = src_tensor.repeat(copy_time+1,1)
        #trg_tensor = trg_tensor.repeat(copy_time+1,1)
        #print(src_tensor)
        #print(trg_tensor)
        b = Transformer.Batch(src_tensor,trg_tensor)
        print('数据最终形态')
        print('输入',b.trg)
        print('输出',b.trg_y)
        print('mask',b.trg_mask)
        return b
    def __len__(self):
        return self.length
    
        

In [22]:
dataset = Mydataset(train_path,tokenizer)
print(len(dataset))
b1 = dataset.__getitem__(1024)

23635
src: 我 真 蠢
trg: I'm so stupid
3
src_id: [[16, 226, 309]]
trg_id: [[1, 23, 175, 698, 2]]
src_tensor: torch.Size([1, 3]) trg_tensor: torch.Size([1, 5])
数据最终形态
输入 tensor([[  1,  23, 175, 698]])
输出 tensor([[ 23, 175, 698,   2]])
mask tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])


将要测试输入输出的模块 实例化步骤

In [23]:
src_vocab,trg_vocab = tokenizer.get_vocab()
d_model = 512
#这是一个简单的Transformer网络，只有一层encoder decoder 注：无生成器
model = Transformer.make_model(src_vocab,trg_vocab,1,d_model)
en_embedding = Transformer.Embeddings(d_model,trg_vocab)
ch_embedding = Transformer.Embeddings(d_model,src_vocab)
multihead_attention = Transformer.MultiHeadedAttention(8,d_model)
pe = Transformer.PositionalEncoding(d_model,0.1,max_len=20)
generater = Transformer.Generator(d_model,trg_vocab)

## Transformer各个结构的输入输出模拟
通过未训练的网络运行数据获得对应的数据形状\
顺序为Embedding->PositionEncoding->MultiheadAttention\
Encoder-MultiHead 输入来源为q:src_tensor k:src_tensor v:src_tensor\
Dncoder-MultiHead\
输入来源为\
1&emsp;q:trg_tensor&emsp;k:trg_tensor&emsp;&emsp;&emsp;v:trg_tensor&emsp;mask:trg_mask\
2&emsp;q:trg_tensor&emsp;k:encoder_output&emsp;v:encoder_output&emsp;mask:src_mask


In [24]:
#这里是模拟一个样本输入Transformer内部数据处理流程，并且打印从各个模块出来的数据形状
src_tensor = ch_embedding(b1.src)
trg_tensor = en_embedding(b1.trg)
print("Embedding：",src_tensor.shape,trg_tensor.shape)
src_tensor = pe(src_tensor)
trg_tensor = pe(trg_tensor)
print("PositionEncoding:",src_tensor.shape,trg_tensor.shape)
print(b1.src_mask.shape)
encoder_output = multihead_attention(src_tensor,src_tensor,src_tensor,b1.src_mask)
print('Encoder-Multihead:',encoder_output.shape)
decoder_output = multihead_attention(trg_tensor,trg_tensor,trg_tensor,b1.trg_mask)
decoder_output = multihead_attention(trg_tensor,encoder_output,encoder_output,b1.src_mask)
print('Decoder-masked-Multihead:',decoder_output.shape)


Embedding： torch.Size([1, 3, 512]) torch.Size([1, 4, 512])
PositionEncoding: torch.Size([1, 3, 512]) torch.Size([1, 4, 512])
torch.Size([1, 1, 3])
Encoder-Multihead: torch.Size([1, 3, 512])
Decoder-masked-Multihead: torch.Size([1, 4, 512])


## Transformer整体输入输出
这里模拟了一条数据应该如何输入Transformer，已经对应的输出应该怎么处理

In [25]:
trg_input = b1.trg
copy_time = trg_input.shape[1]
#print(trg_input.shape)
trg_input = trg_input.repeat(copy_time,1)
src_input = b1.src
src_input = src_input.repeat(copy_time,1)
print('输入模型的数据为(其实就是一句话复制了四遍):\n',src_input,'\n',trg_input)
transformer_output = model(src_input,trg_input,b1.src_mask,b1.trg_mask)
print("Total Model:",transformer_output.shape)
generater_output = generater(transformer_output)
print("Generater:",generater_output.shape)
print("目前我们已经获得了四句话，四个词的id了,当然这四句应该是")
print("I'm |<PAD>|<PAD>|<PAD>")
print("I'm |so | <PAD> |<PAD>")
print("I'm |so |stupid |<PAD>")
print("I'm |so |stupid |<EOS>")
print("那么我们需要将这四句话的id化的向量与generator的输出计算损失")
print("但这里需要注意，每次只计算一个词的损失，如第一行应计算I'm\n\
第二行应计算so 以此类推 其他词的差距不进行计算")

输入模型的数据为(其实就是一句话复制了四遍):
 tensor([[ 16, 226, 309],
        [ 16, 226, 309],
        [ 16, 226, 309],
        [ 16, 226, 309]]) 
 tensor([[  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698]])
Total Model: torch.Size([4, 4, 512])
Generater: torch.Size([4, 4, 8349])
目前我们已经获得了四句话，四个词的id了,当然这四句应该是
I'm |<PAD>|<PAD>|<PAD>
I'm |so | <PAD> |<PAD>
I'm |so |stupid |<PAD>
I'm |so |stupid |<EOS>
那么我们需要将这四句话的id化的向量与generator的输出计算损失
但这里需要注意，每次只计算一个词的损失，如第一行应计算I'm
第二行应计算so 以此类推 其他词的差距不进行计算


## Multihead-Attention 内部数据处理的变化
虽然Transformer中已经实现过一遍了，但这里为了方便展示，添加了一个每个步骤输出数据形状\
方便理解\
这里是代码

In [46]:
def Attention_display(query, key, value, mask=None, dropout=None):
    "普通的点积型注意力"
    #输入的QKV的维度为(batch,head,quelen,d_k)d_k为分头后的数据
    d_k = query.size(-1)
    #只对后两维进行矩阵乘法，保证多头注意力，各个头的数据隔离
    print("A-这时需要进行QK相乘的操作 这时QK的维度为:",query.shape,key.shape)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    print("A-这时已经完成QK相乘的操作，需要对score矩阵进行mask操作 这时Attention score的维度为:",scores.shape)
    if mask is not None:
        #如果有掩码，则将数据替换为-1e9(接近无穷小)
        scores = scores.masked_fill(mask == 0, -1e9)
    print("A-这里如果有掩码操作的话会进行掩码 这时的score维度未变只是数据一部分被遮掩掉了,不方便展示 请看下面的mask单独的演示")
    p_attn = F.softmax(scores, dim = -1)
    print("A-这里对最后一个维度的数据进行softmax操作,此时score的维度为:",scores.shape)
    if dropout is not None:
        p_attn = dropout(p_attn)
    print("A-这里进行了dropout操作维度未发生改变")
    output = torch.matmul(p_attn, value)
    print("A-这进行了将score与value相乘的过程 最终输出为:",output.shape)
    return output, p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "需要输入head数以及d_model,head需要可以整除d_model"
        "多头的目的是让不同的头提取的特征不同，以丰富模型的特征提取"
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # 算出分多头后的维度 如512维 八头维度变为(8,64)
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        #如果有mask操作在第一个维度后加入维度
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 先分别将QKV分别输入线性回归中，然后将得到的QKV分别将维度划分多头
        #shape=(batch,seqlen,d_model)->Linear(维度不变)->(batch,head,seqlen,d_k)
        query, key, value = [l(x) for l, x in zip(self.linears, (query, key, value))]
        print("M-query先进入一个FNN，维度不改变，KV同理  这时query维度:",query.shape)
        query = query.view(nbatches, -1, self.h, self.d_k)
        print("M-这时query进行了分头变换 这时的维度为:",query.shape)
        query = query.transpose(1, 2)
        print("M-这时query进行了维度交换 这时的维度为:",query.shape)
        key = key.view(nbatches, -1, self.h, self.d_k)
        key = key.transpose(1, 2)
        value = value.view(nbatches, -1, self.h, self.d_k)
        value = value.transpose(1, 2)
        # 将分好头的数据输入Attention，并得出结果
        print("M-这时将变形后的QKV以及mask输入Attention")
        x, self.attn = Attention_display(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) 将数据还原为原来的样子 shape=(batch,seqlen,d_model)
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        print("M-这里进行了数据维度的还原 这时维度为:",x.shape)
        print("M-这里对最终的输出输入了一个FNN 维度未改变")
        return self.linears[-1](x)
    
def clones(module, N):
    """这个函数很有用，因为整个网络结构有多次重复结构，可以用这个函数对实例进行深拷贝，例如可以复制多个Encoder或Linear"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [47]:
#实例化一个MultiHeadattention
ma = MultiHeadedAttention(8,512)
#数据使用前面
input_data =  pe(en_embedding(trg_input))
print('MultiHeadAttention 输入:',input_data.shape)
print('对应维度为 (batch，词个数，d_model)')
output = ma(input_data,input_data,input_data,b1.trg_mask)
print('MultiHeadAttention输出',output.shape)

MultiHeadAttention 输入: torch.Size([4, 4, 512])
对应维度为 (batch，词个数，d_model)
M-query先进入一个FNN，维度不改变，KV同理  这时query维度: torch.Size([4, 4, 512])
M-这时query进行了分头变换 这时的维度为: torch.Size([4, 4, 8, 64])
M-这时query进行了维度交换 这时的维度为: torch.Size([4, 8, 4, 64])
M-这时将变形后的QKV以及mask输入Attention
A-这时需要进行QK相乘的操作 这时QK的维度为: torch.Size([4, 8, 4, 64]) torch.Size([4, 8, 4, 64])
A-这时已经完成QK相乘的操作，需要对score矩阵进行mask操作 这时Attention score的维度为: torch.Size([4, 8, 4, 4])
A-这里如果有掩码操作的话会进行掩码 这时的score维度未变只是数据一部分被遮掩掉了,不方便展示 请看下面的mask单独的演示
A-这里对最后一个维度的数据进行softmax操作,此时score的维度为: torch.Size([4, 8, 4, 4])
A-这里进行了dropout操作维度未发生改变
A-这进行了将score与value相乘的过程 最终输出为: torch.Size([4, 8, 4, 64])
M-这里进行了数据维度的还原 这时维度为: torch.Size([4, 4, 512])
M-这里对最终的输出输入了一个FNN 维度未改变
MultiHeadAttention输出 torch.Size([4, 4, 512])


## mask 展示
下面展示以下mask后的decoder输入
这个操作其实是在attention中执行的，这里只是简单展示

In [45]:
print(trg_input)
print("-------------------------------------------------")
print(trg_input.masked_fill(b1.trg_mask==0,-1e9))

tensor([[  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698]])
-------------------------------------------------
tensor([[[          1, -1000000000, -1000000000, -1000000000],
         [          1,          23, -1000000000, -1000000000],
         [          1,          23,         175, -1000000000],
         [          1,          23,         175,         698]]])


## 以上为一条数据全部流程
这里没有演示损失函数计算以及优化过程，损失函数哈佛学习版使用了KL散度，而attention论文中使用了交叉熵，这里不做评价，请自行选择合适的损失函数\
Transformer的优化是带有热身的，1个小时连热身都跑不完，所以就算了\
一条数据的如何处理如何走过整个模型的样子已经演示了\
相信你肯定会批量训练了吧🤡
